# Final Project

### 1.1 Part 1: VGG neural network

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Create VGG model
model = Sequential()
model.add(VGG16(include_top=False, weights='imagenet', input_shape=(32, 32, 3)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test accuracy: {accuracy * 100:.2f}%')


58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/10
625/625 [==============================] - 4260s 7s/step - loss: 2.3068 - accuracy: 0.1000 - val_loss: 2.3026 - val_accuracy: 0.1025
Epoch 2/10
625/625 [==============================] - 4300s 7s/step - loss: 2.3027 - accuracy: 0.0977 - val_loss: 2.3027 - val_accuracy: 0.1014
Epoch 3/10
625/625 [==============================] - 4254s 7s/step - loss: 2.3027 - accuracy: 0.0997 - val_loss: 2.3027 - val_accuracy: 0.0952
Epoch 4/10
132/625 [=====>........................] - ETA: 54:11 - loss: 2.3027 - accuracy: 0.1023

###1.2 Part 2: Residual neural network

In [1]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Define Residual Block
def residual_block(x, filters, kernel_size=3, stride=1):
    shortcut = x

    # First convolution layer
    x = Conv2D(filters, kernel_size, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(filters, kernel_size, strides=1, padding='same')(x)
    x = BatchNormalization()(x)

    # If the input shape and output shape don't match, add a 1x1 conv. to the shortcut
    if K.int_shape(shortcut)[-1] != filters or stride != 1:
        shortcut = Conv2D(filters, 1, strides=stride, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)

    x = Add()([x, shortcut])
    x = ReLU()(x)

    return x

# Build ResNet model
input_tensor = Input(shape=(32, 32, 3))
x = Conv2D(64, 7, strides=2, padding='same')(input_tensor)
x = BatchNormalization()(x)
x = ReLU()(x)
x = residual_block(x, 64)
x = residual_block(x, 64)
x = residual_block(x, 128, stride=2)
x = residual_block(x, 128)
x = residual_block(x, 256, stride=2)
x = residual_block(x, 256)
x = AveragePooling2D(4)(x)
x = Flatten()(x)
output_tensor = Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)

# Train the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test accuracy: {accuracy * 100:.2f}%')


170498071/170498071 [==============================] - 3s 0us/step
Epoch 1/10
625/625 [==============================] - 887s 1s/step - loss: 1.3891 - accuracy: 0.4989 - val_loss: 1.3789 - val_accuracy: 0.5242
Epoch 2/10
625/625 [==============================] - 880s 1s/step - loss: 0.9646 - accuracy: 0.6576 - val_loss: 1.4382 - val_accuracy: 0.5238
Epoch 3/10
625/625 [==============================] - 883s 1s/step - loss: 0.7495 - accuracy: 0.7376 - val_loss: 0.9497 - val_accuracy: 0.6770
Epoch 4/10
625/625 [==============================] - 878s 1s/step - loss: 0.6054 - accuracy: 0.7878 - val_loss: 0.8563 - val_accuracy: 0.7193
Epoch 5/10
625/625 [==============================] - 876s 1s/step - loss: 0.4797 - accuracy: 0.8299 - val_loss: 1.0989 - val_accuracy: 0.6708
Epoch 6/10
625/625 [==============================] - 876s 1s/step - loss: 0.3718 - accuracy: 0.8698 - val_loss: 1.1072 - val_accuracy: 0.6793
Epoch 7/10
625/625 [==============================] - 878s 1s/step - loss: 